In [1]:
import os
import pandas as pd

# Load CSV file
csv_file_path = r"C:\Users\aks\Desktop\zigguratss\metadata_sorted.xlsx"
data = pd.read_excel(csv_file_path)



In [2]:
data.head()

,image_id,description
0,image3935.jpeg,a painting on a wall with a bunch of different...
1,image3937.jpeg,a painting of a girl with a cat on her head
2,image3938.jpeg,a statue of a bird on a leafy plant
3,image3939.jpeg,a painting of a horse in the middle of a field
4,image3941.jpeg,a collage of photos of people and animals


In [3]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
from PIL import Image
import torch

# Load the model
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

def generate_caption(image_path):
    # Open image
    image = Image.open(image_path).convert("RGB")
    # Preprocess the image
    pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values

    # Generate caption
    with torch.no_grad():
        output_ids = model.generate(pixel_values)
    # Decode the generated caption
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return caption

# Ensure image folder path is set correctly
images_folder = 'C:\\Users\\aks\\Desktop\\zigguratss\\artwork'  # Update this path as needed

# Add a new column for descriptions
data['Generated_Description'] = ""

# Process each row in the dataframe
for index, row in data.iterrows():
    image_path = os.path.join(images_folder, row['image_id'].strip())

    # Check if the file exists
    if os.path.exists(image_path):
        description = generate_caption(image_path)
        data.at[index, 'Generated_Description'] = description
    else:
        print(f"File not found: {image_path}")

# Save the updated DataFrame to a new CSV file
output_csv_file_path = 'output_csv_file.csv'
data.to_csv(output_csv_file_path, index=False)
print(f"Descriptions saved to {output_csv_file_path}")


c:\Users\aks\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\aks\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
c:\Users\aks\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. W

Descriptions saved to output_csv_file.csv
